<a href="https://colab.research.google.com/github/TimotheeeNiven/IntroML_TNiven/blob/main/Homework7_TNiven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import time
import datetime


In [2]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)



170498071/170498071 [==============================] - 6s 0us/step


In [ ]:
# Build CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))  # Adjust the number of output classes

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
epochs = 300
start_time = time.time()
history = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))
end_time = time.time()

Epoch 1/300
1563/1563 [==============================] - 82s 51ms/step - loss: 1.4926 - accuracy: 0.4557 - val_loss: 1.2482 - val_accuracy: 0.5485
Epoch 2/300
1563/1563 [==============================] - 76s 49ms/step - loss: 1.1462 - accuracy: 0.5931 - val_loss: 1.0868 - val_accuracy: 0.6165
Epoch 3/300
1563/1563 [==============================] - 76s 49ms/step - loss: 1.0136 - accuracy: 0.6446 - val_loss: 0.9993 - val_accuracy: 0.6495
Epoch 4/300
1563/1563 [==============================] - 78s 50ms/step - loss: 0.9215 - accuracy: 0.6775 - val_loss: 0.9555 - val_accuracy: 0.6664
Epoch 5/300
1563/1563 [==============================] - 76s 49ms/step - loss: 0.8441 - accuracy: 0.7046 - val_loss: 0.9160 - val_accuracy: 0.6801
Epoch 6/300
1563/1563 [==============================] - 76s 49ms/step - loss: 0.7809 - accuracy: 0.7256 - val_loss: 0.8855 - val_accuracy: 0.6926
Epoch 7/300
1563/1563 [==============================] - 79s 50ms/step - loss: 0.7259 - accuracy: 0.7440 - val_loss: 0

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)

# Print results
print(f"Training Time: {end_time - start_time} seconds")
print(f"Training Loss after {epochs} epochs: {history.history['loss'][-1]}")
print(f"Test Accuracy after {epochs} epochs: {test_acc}")

313/313 [==============================] - 4s 12ms/step - loss: 11.7568 - accuracy: 0.6734
Training Time: 22285.848536491394 seconds
Training Loss after 300 epochs: 0.11062169820070267
Test Accuracy after 300 epochs: 0.6733999848365784


In [ ]:
#Problem 1B
# Build extended CNN model
extended_model = models.Sequential()
extended_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
extended_model.add(layers.MaxPooling2D((2, 2)))
extended_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
extended_model.add(layers.MaxPooling2D((2, 2)))
extended_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# Additional layer
extended_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
extended_model.add(layers.MaxPooling2D((2, 2)))
extended_model.add(layers.Flatten())
extended_model.add(layers.Dense(128, activation='relu'))
extended_model.add(layers.Dense(10, activation='softmax'))

# Compile the extended model
extended_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])


In [ ]:
# Train the extended model for 300 epochs
start_time = time.time()
extended_model.fit(x_train, y_train, epochs=300, validation_data=(x_test, y_test))
training_time = time.time() - start_time

Epoch 1/300


KeyboardInterrupt: ignored

In [3]:
#Problem 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Load CIFAR-10 dataset and define data loaders
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:08<00:00, 20922375.30it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
#In class edition Defining th Module
class ResNet10(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.4)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)  # 10 classes for CIFAR-10

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [6]:
# Initialize the model
model = ResNet10().to(device)

# Loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

In [7]:
# Training loop
epochs = 300
def training_loop_reg(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            # L2 regularization
            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum() for name, p in model.named_parameters() if 'weight' in name)
            loss = loss + l2_lambda * l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()
        if epoch == 1 or epoch % 5 == 0:
            print('{} Epoch {}, Training loss: {}'.format(datetime.datetime.now(), epoch, loss_train / len(train_loader)))

        # Evaluation phase
        if epoch % 5 == 0:  # You can adjust the frequency of evaluation
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for val_imgs, val_labels in val_loader:
                    val_imgs = val_imgs.to(device=device)
                    val_labels = val_labels.to(device=device)
                    val_outputs = model(val_imgs)
                    _, predicted = torch.max(val_outputs, 1)
                    total += val_labels.size(0)
                    correct += (predicted == val_labels).sum().item()

            accuracy = correct / total
            print('{} Epoch {}, Validation Accuracy: {}'.format(datetime.datetime.now(), epoch, accuracy))
  # Get evaluation accuracy at the end
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for val_imgs, val_labels in val_loader:
            val_imgs = val_imgs.to(device=device)
            val_labels = val_labels.to(device=device)
            val_outputs = model(val_imgs)
            _, predicted = torch.max(val_outputs, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()

    final_accuracy = correct / total
    print('Training finished. Final Evaluation Accuracy: {}'.format(final_accuracy))

In [9]:
# Run the training loop
training_loop_reg(epochs, optimizer, model, loss_fn, trainloader, testloader)

2023-12-12 22:36:20.214757 Epoch 1, Training loss: 2.3253070222752172
2023-12-12 22:37:54.022886 Epoch 5, Training loss: 1.7044303339460622
2023-12-12 22:37:59.074974 Epoch 5, Validation Accuracy: 0.4108
2023-12-12 22:40:00.574102 Epoch 10, Training loss: 1.4944721629552524
2023-12-12 22:40:04.517320 Epoch 10, Validation Accuracy: 0.4788
2023-12-12 22:42:01.338265 Epoch 15, Training loss: 1.3644003807888616
2023-12-12 22:42:05.185264 Epoch 15, Validation Accuracy: 0.5375
2023-12-12 22:44:02.217979 Epoch 20, Training loss: 1.2652982959661947
2023-12-12 22:44:06.666514 Epoch 20, Validation Accuracy: 0.5651
2023-12-12 22:46:05.002390 Epoch 25, Training loss: 1.212263840391203
2023-12-12 22:46:10.140880 Epoch 25, Validation Accuracy: 0.5934
2023-12-12 22:48:09.215808 Epoch 30, Training loss: 1.1678725481795533
2023-12-12 22:48:13.296908 Epoch 30, Validation Accuracy: 0.6172
2023-12-12 22:50:10.580078 Epoch 35, Training loss: 1.1379103059201594
2023-12-12 22:50:14.494315 Epoch 35, Validatio

In [11]:
#Problem 2B
# Define the model with Weight Decay, Dropout, and Batch Normalization
class NetRes2B(nn.Module):
    def __init__(self, n_chans1=32, dropout_prob=0.3):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_bn = nn.BatchNorm2d(n_chans1)
        self.conv1_dropout = nn.Dropout2d(p=dropout_prob)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(n_chans1 // 2)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2, kernel_size=3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(n_chans1 // 2)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)  # 10 classes for CIFAR-10

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1_bn(self.conv1(x))), 2)
        out = F.max_pool2d(torch.relu(self.conv2_bn(self.conv2(out))), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3_bn(self.conv3(out))) + out1, 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [16]:
# Initialize the model with Weight Decay, Dropout, and Batch Normalization
epochs = 300
model2B = NetRes2B(dropout_prob=0.3).to(device)
loss_fn2B = nn.CrossEntropyLoss()
optimizer2B = optim.SGD(model2B.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)

In [17]:
print("Training with Weight Decay:")
training_loop_reg(epochs, optimizer2B, model2B, loss_fn2B, trainloader, testloader)

Training with Weight Decay:
2023-12-13 00:41:56.066238 Epoch 1, Training loss: 1.872602863232498
2023-12-13 00:43:27.668191 Epoch 5, Training loss: 1.3076506555842622
2023-12-13 00:43:31.496421 Epoch 5, Validation Accuracy: 0.584


KeyboardInterrupt: ignored

In [12]:
model2b = NetRes2B(dropout_prob=0.3).to(device)

# Loss function and optimizer with Dropout
loss_fn2b = nn.CrossEntropyLoss()
optimizer2b = optim.SGD(model2b.parameters(), lr=0.001, momentum=0.9)

In [14]:
# Training loop
print("\nTraining with Batch Normalization:")
training_loop_reg(epochs, optimizer2b, model2b, loss_fn2b, trainloader, testloader)


Training with Batch Normalization:
2023-12-13 00:39:40.087138 Epoch 1, Training loss: 1.922074217015825
2023-12-13 00:41:13.853167 Epoch 5, Training loss: 1.3011005293682714
2023-12-13 00:41:17.777779 Epoch 5, Validation Accuracy: 0.5914


KeyboardInterrupt: ignored